In [1]:
!pip install -q langchain datasets langchain-google-genai langchain-community sentence_transformers unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━

In [ ]:
!git clone https://github.com/ShishirPatil/gorilla.git

Cloning into 'gorilla'...
remote: Enumerating objects: 1868, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 1868 (delta 1), reused 5 (delta 0), pack-reused 1857
Receiving objects: 100% (1868/1868), 189.60 MiB | 21.87 MiB/s, done.
Resolving deltas: 100% (984/984), done.
Updating files: 100% (337/337), done.


In [ ]:
%%capture
!pip install -r gorilla/raft/requirements.txt

In [ ]:
%cd gorilla/raft

/content/gorilla/raft


In [ ]:
from tqdm.notebook import tqdm
import os
import pandas as pd
from typing import Optional, List, Tuple
import json

from datasets import Dataset

pd.set_option("display.max_colwidth", None)

In [ ]:
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
HF_TOKEN=userdata.get('HF_TOKEN')

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## Generate Synthetic Test Set

### Documents Loading

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
document_folder = "../../gdrive/MyDrive/LLM01 - RắnGPT/Data/Data as Documents"

In [ ]:
import google.generativeai as genai
genai.configure(api_key=GOOGLE_API_KEY)

gemini = genai.GenerativeModel('gemini-pro')

def call_llm(model, question):

    return model.generate_content(question).text

In [ ]:
from langchain_google_genai import GoogleGenerativeAI

langchain_llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY)

In [ ]:
# https://python.langchain.com/docs/integrations/text_embedding/bge_huggingface/
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-m3"
model_kwargs = {"device": device} # Use CPU to avoid RateLimitExceed error in testset generation
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

### Rewrite RAFT Code

In [ ]:
from typing import Any
from dotenv import load_dotenv
# from langchain_openai import OpenAIEmbeddings, AzureOpenAIEmbeddings
from openai import AzureOpenAI, OpenAI
import logging
from os import environ

logger = logging.getLogger("client_utils")

load_dotenv()  # take environment variables from .env.

def build_openai_client(**kwargs: Any):
    """
    Build OpenAI client based on the environment variables.
    """
    client = genai.GenerativeModel('gemini-pro')
    return client

def build_langchain_embeddings(**kwargs: Any):
    """
    Build OpenAI embeddings client based on the environment variables.
    """

    env = read_env_config("EMBEDDING")

    model_name = "BAAI/bge-m3"
    model_kwargs = {"device": device} # Use CPU to avoid RateLimitExceed error in testset generation
    encode_kwargs = {"normalize_embeddings": True}
    client = HuggingFaceBgeEmbeddings(
        model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
    )

    return client

def is_azure():
    azure = "AZURE_OPENAI_ENDPOINT" in environ or "AZURE_OPENAI_KEY" in environ or "AZURE_OPENAI_AD_TOKEN" in environ
    if azure:
        logger.debug("Using Azure OpenAI environment variables")
    else:
        logger.debug("Using OpenAI environment variables")
    return azure

In [ ]:
safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]

In [ ]:
import mdc
from mdc import MDC
from logconf import log_setup
import logging
from typing import Literal, Any
import argparse
from openai import OpenAI
import datasets
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer
import json
import PyPDF2
import random
import os, shutil
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from math import ceil
from format import DatasetConverter, datasetFormats, outputDatasetTypes

from langchain.document_loaders import DirectoryLoader

log_setup()

logger = logging.getLogger("raft")

DocType = Literal["api", "pdf", "json", "txt"]

import argparse
from types import SimpleNamespace
import random
def get_chunks(
    file_path: str,
    doctype: DocType = "pdf",
    chunk_size: int = 512,
    openai_key: str | None = None,
    model: str = None
) -> list[str]:
    """
    Takes in a `file_path` and `doctype`, retrieves the document, breaks it down into chunks of size
    `chunk_size`, and returns the chunks.
    """
    chunks = []

    logger.info(f"Retrieving chunks from {file_path} of type {doctype}")

    if doctype == "api":
        pass

    else:

        loader = DirectoryLoader(file_path)
        documents = loader.load()
        
        chunks = [chunk.page_content for chunk in documents]

        # random.shuffle(chunks)
        chunks = chunks[50:]

        logger.info(f"Splitting text into {len(chunks)} chunks using the {model} model.")

    return chunks

def generate_instructions(client: OpenAI, api_call: Any, x=5, model: str = None) -> list[str]:
    """
    Generates `x` questions / use cases for `api_call`. Used when the input document is of type `api`.
    """
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a synthetic instruction-api pair generator. Given an API endpoint in the form of a JSON object, generate %s example queries of instructions a user could ask and would be answered by invoking the API call. For example, if the given API call is the `service.users().getProfile(userId='me').execute()` call from the Gmail API, an example query could be 'How can I fetch my Gmail account's email address?'" % (x)},
            {"role": "system", "content": "The API endpoint is a JSON object with required params: user_name, api_name, api_call, api_version, api_arguments, functionality, and optional params: env_requirements, example_code, meta_data, Questions"},
            {"role": "system", "content": "For instance, if the api call contains: {'user_name': 'felixzhu555', 'api_name': 'Google Maps - Address Validation', 'api_call': 'Client.addressvalidation(addressLines, regionCode=region_code, locality=locality, enableUspsCass=boolean)', 'api_version': '4.10.0', 'api_arguments': {}, 'functionality': 'Validate an address and its components, standardize the address for mailing, and determine the best known geocode for it.', 'env_requirements': ['googlemaps'], 'example_code': 'client = googlemaps.Client(key='YOUR_API_KEY')\nresponse = client.addressvalidation('1600 Amphitheatre Pk', regionCode='US', locality='Mountain View', enableUspsCass=True)', 'meta_data': {'description': 'The googlemaps python client is an abstraction for the Google Maps API that requires python 3.5+. Each Google Maps web service request requires an API key or client ID. API keys are generated in the 'Credentials' page of the 'APIs & Services' tab of Google Cloud console. This key should be kept secret on your server.'}, 'questions': []}, an example instruction would be 'Validate the following address: University Avenue and, Oxford St, Berkeley, CA 94720.'"},
            {"role": "system", "content": "Don't mention 'API' or use any hints or the name of the API. In one-third of the queries, make sure to include a specific example, like 'Validate this address: 123 Harrison St, Oakland CA'. Include ONLY the queries in your response."},
            {"role": "user", "content": str(api_call)}
        ]
    )

    queries = response.choices[0].message.content.split('\n')
    queries = [strip_str(q) for q in queries]
    queries = [q for q in queries if any(c.isalpha() for c in q)]

    return queries

def generate_instructions_gen(client, chunk: Any, x: int = 5, model: str = None) -> list[str]:
    """
    Generates `x` questions / use cases for `chunk`. Used when the input document is of general types
    `pdf`, `json`, or `txt`.
    """

    response = client.generate_content(
        f"""
You are a synthetic question-answer pair generator.

Context: {chunk}\n

Given a chunk of context about some topic(s), generate {x} example questions a user could ask in Vietnamese and would be answered using information from the chunk.
For example, if the given context was a Wikipedia paragraph about the United States, an example question could be 'How many states are in the United States?'
The questions should be able to be answered in a few words or less.
If the context is about a specific snake, must mention the snake name in the question and do not use 'loài rắn này' or 'this snake' in the question.  Include only the questions in your response and it must be in Vietnamese.

Provide your answer as follows, in Vietnamese:

Output:::
question_1\nquestion_2\n...

Output:::
        """, safety_settings=safety_settings
    )

    queries = response.text.split('\n')
    queries = [strip_str(q) for q in queries]
    queries = [q for q in queries if any(c.isalpha() for c in q)]

    return queries

def strip_str(s: str) -> str:
    """
    Helper function for helping format strings returned by GPT-4.
    """
    l, r = 0, len(s)-1
    beg_found = False
    for i in range(len(s)):
        if s[i].isalpha():
            if not beg_found:
                l = i
                beg_found = True
            else:
                r = i
    r += 2
    return s[l:min(r, len(s))]

def encode_question(question: str, api: Any) -> list[str]:
    """
    Encode multiple prompt instructions into a single string for the `api` case.
    """
    prompts = []

    prompt = question + "\nWrite a python program to call API in " + str(api) + ".\n\nThe answer should follow the format: <<<domain>>> $DOMAIN \n, <<<api_call>>>: $API_CALL \n, <<<api_provider>>>: $API_PROVIDER \n, <<<explanation>>>: $EXPLANATION \n, <<<code>>>: $CODE}. Here are the requirements:\n \n2. The $DOMAIN should be the domain of the API ('N/A' if unknown). The $API_CALL should have only 1 line of code that calls api.\n3. The $API_PROVIDER should be the programming framework used.\n4. $EXPLANATION should be a numbered, step-by-step explanation.\n5. The $CODE is the python code.\n6. Do not repeat the format in your answer."
    prompts.append({"role": "system", "content": "You are a helpful API writer who can write APIs based on requirements."})
    prompts.append({"role": "user", "content": prompt})
    return prompts

def encode_question_gen(question: str, chunk: Any) -> list[str]:
    """
    Encode multiple prompt instructions into a single string for the general case (`pdf`, `json`, or `txt`).
    """

    prompts = []
    system_prompt = "You are a dedicated teacher in Snake studies who can provide an answer in Vietnamese given a question and relevant context."
    user_prompt = """
        Question: {question}\nContext: {context}\n
        Answer this question in Vietnamese in an educated manner using the information given in the Context above. Here are things to pay attention to:
        - First provide step-by-step reasoning on how to answer the question. Start with Step 1.
        - In the reasoning, if you need to copy paste some sentences from the context, include them in ##begin_quote## and ##end_quote##. This would mean that things outside of ##begin_quote## and ##end_quote## are not directly copy paste from the context.
        - End your response with final answer in the form <ANSWER>: $answer. The answer should not be too short and it must add additional information from the context that could help user learn more than what they expect from the question.
        - For example, if the question is "Rắn có thể chết vì những nguyên nhân nào trong thời kỳ ngủ đông?", then answer "Có nhiều nguyên nhân dẫn đến rắn chết trong thời kì ngủ đông. Chúng có thể trở thành con mồi cho các loài động vật khác như chồn, gấu trúc và cú, hoặc cũng có thể chết vì lạnh hoặc đói nếu chúng không chuẩn bị đủ cho mùa đông."
        - If you can not find enough information to answer the question or you are unsure, then politely say "I don't have the answer based on my knowledge" and stop there. Do not mention the phrases "ngữ cảnh" or "đoạn văn" in your answer.
        You MUST provide answer and reasonings in Vietnamese, and begin your final answer with the English tag "<ANSWER>:".
    """.format(question=question, context=str(chunk))

    return f"{system_prompt}\n{user_prompt}"

def generate_label(client, question: str, context: Any, doctype: DocType = "pdf", model: str = None) -> str | None:
    """
    Generates the label / answer to `question` using `context` and GPT-4.
    """
    question = encode_question(question, context) if doctype == "api" else encode_question_gen(question, context)

    try:
        response = client.generate_content(question, safety_settings=safety_settings).text
    except Exception as e:
        print(str(e))
        response = ""
    return response

def add_chunk_to_dataset(
    client: OpenAI,
    chunks: list[str],
    chunk: str,
    doctype: DocType = "api",
    x: int = 5,
    num_distract: int = 3,
    p: float = 0.8,
    model: str = None
) -> None:
    """
    Given a chunk, create {Q, A, D} triplets and add them to the dataset.
    """
    global ds
    i = chunks.index(chunk)
    qs = generate_instructions(client, chunk, x, model) if doctype == "api" else generate_instructions_gen(client, chunk, x, model)
    for q in qs:
        datapt = {
            "id": None,
            "type": None,
            "question": None,
            "context": None,
            "oracle_context": None,
            "cot_answer": None
        }

        datapt["id"] = f"seed_task_{0 if not ds else ds.num_rows}"
        datapt["type"] = "api call" if doctype == "api" else "general"
        datapt["question"] = q

        # add num_distract distractor docs
        docs = [chunk]
        indices = list(range(0, len(chunks)))
        indices.remove(i)
        for j in random.sample(indices, num_distract):
            docs.append(chunks[j])
        # decides whether to add oracle document
        oracle = random.uniform(0, 1) < p
        if not oracle:
            docs[0] = chunks[random.sample(indices, 1)[0]]
        random.shuffle(docs)

        d = {
            "title": [],
            "sentences": []
        }

        d["title"].append(["placeholder_title"]*(num_distract+1))
        d["sentences"].append(docs)
        datapt["context"] = d
        datapt["oracle_context"] = chunk

        # add answer to q
        datapt["cot_answer"] = generate_label(client, q, chunk, doctype, model=model)

        # construct model instruction
        context = ""
        for doc in docs:
            context += "<DOCUMENT>" + str(doc) + "</DOCUMENT>\n"
        context += q
        datapt["instruction"] = context

        # add to dataset
        if not ds:
            # init ds
            datapt["id"] = [datapt["id"]]
            datapt["type"] = [datapt["type"]]
            datapt["question"] = [datapt["question"]]
            datapt["context"] = [datapt["context"]]
            datapt["oracle_context"] = [datapt["oracle_context"]]
            datapt["cot_answer"] = [datapt["cot_answer"]]
            datapt["instruction"] = [datapt["instruction"]]
            ds = Dataset.from_dict(datapt)
        else:
            ds = ds.add_item(datapt)

def save_checkpoint(state, filename):
    with open(filename, 'w') as f:
        f.write(str(state))

def load_checkpoint(filename):
    with open(filename, 'r') as f:
        return int(f.read())

from time import sleep
# Every N chunks, save checkpoint
N = 10

def main():
    global ds

    # run code
    args = get_args()

    # Validate arguments
    if args.output_chat_system_prompt and args.output_format != "chat":
        raise Exception("Parameter --output-chat-system-prompt can only be used with --output-format chat")

    OPENAPI_API_KEY = args.openai_key

    # client = build_openai_client(
    #     api_key=OPENAPI_API_KEY,
    # )

    client = genai.GenerativeModel('gemini-pro')

    CHUNK_SIZE = args.chunk_size
    NUM_DISTRACT_DOCS = args.distractors

    chunks = get_chunks(args.datapath, args.doctype, CHUNK_SIZE, OPENAPI_API_KEY, model=args.embedding_model)

    ds = None

    num_chunks = len(chunks)

    if not args.fast:
        start = 0
        if os.path.exists("checkpoint.txt"):
            start = int(load_checkpoint("checkpoint.txt"))

        for i in range((start//N)*N, len(chunks)):
            chunk = chunks[i]
            save_checkpoint(i, "checkpoint.txt")

            perc = ceil(i / num_chunks * 100)
            with MDC(progress=f"{perc}%"):
                logger.info(f"Adding chunk {i}/{num_chunks}")
                add_chunk_to_dataset(client, chunks, chunk, args.doctype, args.questions, NUM_DISTRACT_DOCS, model=args.completion_model)

            if (i+1) % N == 0:
                ds.save_to_disk(args.output + "-checkpoints-" + str(i))
                ds = None

            sleep(5)


        if ds:
            ds.save_to_disk(args.output + "-checkpoints-last")

        ds_list = []

        for filename in os.listdir(os.path.dirname(args.output)):
            if "-checkpoints-" in filename:
                for f in os.listdir(os.path.dirname(args.output) + "/" + filename):
                    if f.endswith(".arrow"):
                        ds_list.append(Dataset.from_file(os.path.dirname(args.output) + "/" + filename + "/" + f))

        ds = datasets.concatenate_datasets(ds_list)
    else:
        for i, chunk in enumerate(chunks):
            perc = ceil(i / num_chunks * 100)
            with MDC(progress=f"{perc}%"):
                logger.info(f"Adding chunk {i}/{num_chunks}")
                add_chunk_to_dataset(client, chunks, chunk, args.doctype, args.questions, NUM_DISTRACT_DOCS, model=args.completion_model)

    # Save as .arrow format
    ds.save_to_disk(args.output)

    # Save as .jsonl format
    formatter = DatasetConverter()

    # Extract format specific params
    format_params = {}
    if args.output_chat_system_prompt:
        format_params['system_prompt'] = args.output_chat_system_prompt

    formatter.convert(ds=ds, format=args.output_format, output_path=args.output, output_type=args.output_type, params=format_params)

    if not args.fast:
        os.remove("checkpoint.txt")
        for filename in os.listdir(os.path.dirname(args.output)):
            if "-checkpoints-" in filename:
                shutil.rmtree(os.path.dirname(args.output) + "/" + filename)

In [ ]:
system_prompt = "Bạn là một trợ lí Tiếng Việt nhiệt tình và trung thực. Hãy luôn trả lời một cách hữu ích nhất có thể, đồng thời giữ an toàn.\n"
system_prompt += "Câu trả lời của bạn không nên chứa bất kỳ nội dung gây hại, phân biệt chủng tộc, phân biệt giới tính, độc hại, nguy hiểm hoặc bất hợp pháp nào. Hãy đảm bảo rằng các câu trả lời của bạn không có thiên kiến xã hội và mang tính tích cực."
system_prompt += "Nếu một câu hỏi không có ý nghĩa hoặc không hợp lý về mặt thông tin, hãy giải thích tại sao thay vì trả lời một điều gì đó không chính xác. Nếu bạn không biết câu trả lời cho một câu hỏi, hãy trẳ lời là bạn không biết và vui lòng không chia sẻ thông tin sai lệch."

def get_args() -> SimpleNamespace:
    """
    Returns a namespace with predefined default values for arguments
    """
    default_args = {
        "datapath": "../../gdrive/MyDrive/LLM01 - RắnGPT/Data/Data as Documents",
        "output": "../../",
        "output_format": "chat",
        "output_type": "jsonl",
        "output_chat_system_prompt": system_prompt,
        "distractors": 4,
        "p": 0.8,
        "questions": 5,
        "chunk_size": 512,
        "doctype": "txt",
        "openai_key": None,
        "embedding_model": None,
        "completion_model": None,
        "fast": False,
    }

    return SimpleNamespace(**default_args)


In [ ]:
main()

2024-05-27 02:28:01  INFO [    ] raft Retrieving chunks from ../../gdrive/MyDrive/LLM01 - RắnGPT/Data/Data as Documents of type txt
2024-05-27 02:28:08  INFO [    ] raft Splitting text into 132 chunks using the None model.
2024-05-27 02:28:08  INFO [  0%] raft Adding chunk 0/132
The `response.text` quick accessor only works when the response contains a valid `Part`, but none was returned. Check the `candidate.safety_ratings` to see if the response was blocked.
2024-05-27 02:28:39  INFO [  1%] raft Adding chunk 1/132
2024-05-27 02:29:06  INFO [  2%] raft Adding chunk 2/132
2024-05-27 02:29:34  INFO [  3%] raft Adding chunk 3/132
2024-05-27 02:30:05  INFO [  4%] raft Adding chunk 4/132
2024-05-27 02:30:14  INFO [  4%] raft Adding chunk 5/132
2024-05-27 02:30:47  INFO [  5%] raft Adding chunk 6/132
2024-05-27 02:31:15  INFO [  6%] raft Adding chunk 7/132
2024-05-27 02:31:44  INFO [  7%] raft Adding chunk 8/132
2024-05-27 02:32:16  INFO [  7%] raft Adding chunk 9/132
2024-05-27 02:32:52 

Saving the dataset (0/1 shards):   0%|          | 0/651 [00:00<?, ? examples/s]

Map:   0%|          | 0/651 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from datasets import Dataset

ds = Dataset.from_file("/content/-checkpoints-39/data-00000-of-00001.arrow")
print(ds)
ds.to_pandas()

Dataset({
    features: ['id', 'type', 'question', 'context', 'oracle_context', 'cot_answer', 'instruction'],
    num_rows: 47
})


id     type  \
0    seed_task_0  general   
1    seed_task_1  general   
2    seed_task_2  general   
3    seed_task_3  general   
4    seed_task_4  general   
5    seed_task_5  general   
6    seed_task_6  general   
7    seed_task_7  general   
8    seed_task_8  general   
9    seed_task_9  general   
10  seed_task_10  general   
11  seed_task_11  general   
12  seed_task_12  general   
13  seed_task_13  general   
14  seed_task_14  general   
15  seed_task_15  general   
16  seed_task_16  general   
17  seed_task_17  general   
18  seed_task_18  general   
19  seed_task_19  general   
20  seed_task_20  general   
21  seed_task_21  general   
22  seed_task_22  general   
23  seed_task_23  general   
24  seed_task_24  general   
25  seed_task_25  general   
26  seed_task_26  general   
27  seed_task_27  general   
28  seed_task_28  general   
29  seed_task_29  general   
30  seed_task_30  general   
31  seed_task_31  general   
32  seed_task_32  general   
33  seed_task_33  general   
34  seed_task_34  general   
35  seed_task_35  general   
36  seed_task_36  general   
37  seed_task_37  general   
38  seed_task_38  general   
39  seed_task_39  general   
40  seed_task_40  general   
41  seed_task_41  general   
42  seed_task_42  general   
43  seed_task_43  general   
44  seed_task_44  general   
45  seed_task_45  general   
46  seed_task_46  general   

                                                                                     question  \
0                                               Rắn nào thích nhiệt độ ưa thích là bao nhiêu?   
1                                                              Cách tăng độ ẩm cho rắn là gì?   
2                                                               Rắn cần những nơi trú ẩn nào?   
3                                                     Rắn hoàng boa có chế độ ăn như thế nào?   
4                                        Nên cách bao lâu thì cần vệ sinh chuồng rắn một lần?   
5                                          Thức ăn phù hợp với rắn cảnh trăn Miến Điện là gì?   
6                                             Thức ăn cho rắn cảnh trăn Mộc Châu là loài nào?   
7                                                             Rắn cảnh rắn nước có thể ăn gì?   
8                                                Thức ăn cho rắn cảnh rắn hổ mang chúa là gì?   
9                                    Rắn cảnh trăn hoàng gia có thể ăn được loại thức ăn nào?   
10     Nuôi một con rắn thú cưng hợp pháp trong khu vực của tôi có những yêu cầu pháp lý nào?   
11                                                  Thú cưng phải được cung cấp từ nguồn nào?   
12                                        Mục đích của việc đảm bảo nguồn gốc vật nuôi là gì?   
13                                          Việc nuôi thú cưng từ nguồn nào là bị nghiêm cấm?   
14                                           Việc tuân thủ các yêu cầu pháp lý có lợi ích gì?   
15                                Làm thế nào để tìm được nhà lai tạo có uy tín cho rắn cảnh?   
16                  Những trang mạng xã hội nào có thể tìm được thông tin nhà lai tạo uy tín?   
17                                        Rắn nào có nguy cơ tuyệt chủng do biến đổi khí hậu?   
18                        Nhiệt độ ấm hơn có thể có tác động như thế nào đến hành vi của rắn?   
19             Sự gia tăng nhiệt độ có thể ảnh hưởng đến nhu cầu thức ăn của rắn như thế nào?   
20                  Biến đổi khí hậu có thể làm thay đổi tỷ lệ giới tính của rắn như thế nào?   
21                          Biến đổi khí hậu có thể ảnh hưởng đến số lượng rắn theo cách nào?   
22                                                              Đ mối đe dọa nào đối với rắn?   
23                                       Tại sao săn bắt rắn là một mối đe dọa đối với chúng?   
24                                            Nạn săn bắt rắn hoang dã có bất hợp pháp không?   
25                                         Việc săn bắt rắn hoang dã phục vụ cho mục đích gì?   
26

In [ ]:
from datasets import Dataset, concatenate_datasets

ds1 = Dataset.from_file("/content/data-00000-of-00001.arrow")
ds2 = Dataset.from_file("/content/checkpoints-49/data-00000-of-00001.arrow")
ds3 = Dataset.from_file("/content/-checkpoints-9/data-00000-of-00001.arrow")
ds4 = Dataset.from_file("/content/-checkpoints-19/data-00000-of-00001.arrow")
ds5 = Dataset.from_file("/content/-checkpoints-29/data-00000-of-00001.arrow")
ds6 = Dataset.from_file("/content/-checkpoints-39/data-00000-of-00001.arrow")

dataset_cc = concatenate_datasets([ds1, ds2, ds3, ds4, ds5, ds6])

print(dataset_cc)
dataset_cc.to_pandas().head(1)

Dataset({
    features: ['id', 'type', 'question', 'context', 'oracle_context', 'cot_answer', 'instruction'],
    num_rows: 885
})


id     type                                question  \
0  seed_task_0  general  Rắn đuôi đỏ kiếm ăn vào thời điểm nào?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [ ]:
# Remove those whose questions contain "loài rắn này"

exclude_idx = [11, 48, 68, 88, 133, 158, 188, 191, 350, 352, 354, 417, 597, 598, 599, 600, 601, ]

# create new dataset exluding those idx
dataset_cc = dataset_cc.select(
    [
        i for i in range(len(dataset_cc))
        if (i + 1) not in set(exclude_idx)
    ]

)


In [ ]:
# Split into train/test

dataset_cc = dataset_cc.train_test_split(test_size=0.2)

In [ ]:
# Push to hub
dataset_cc.push_to_hub("panda1835/rangpt-raft-chat", token=HF_TOKEN)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/610 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/panda1835/rangpt-raft-chat/commit/e3032f18151c0f09e4b3b2957680dc73bc702369', commit_message='Upload dataset', commit_description='', oid='e3032f18151c0f09e4b3b2957680dc73bc702369', pr_url=None, pr_revision=None, pr_num=None)

## Run from script

In [ ]:
!python3 raft.py \
  --datapath "../../gdrive/MyDrive/LLM01 - RắnGPT/Data/Data as Documents/all_1.txt" \
  --output-chat-system-prompt "Hello" \
  --output "../.." \
  --output-format chat \
  --output-type jsonl \
  --distractors 3 \
  --p 0.8 \
  --doctype txt \
  --chunk_size 512 \
  --questions 2 \
  --openai_key $OPENAI_API_KEY \
  --completion_model "gpt-4o" \
  --embedding_model "text-embedding-ada-002"


2024-05-23 14:33:06  INFO [  0%] raft Retrieving chunks from ../../gdrive/MyDrive/LLM01 - RắnGPT/Data/Data as Documents/all_1.txt of type txt
2024-05-23 14:33:06  INFO [  0%] raft Splitting text into 5 chunks using the text-embedding-ada-002 model.
2024-05-23 14:33:07  INFO [  0%] raft Adding chunk 0/5
2024-05-23 14:33:21  INFO [ 20%] raft Adding chunk 1/5
2024-05-23 14:33:33  INFO [ 40%] raft Adding chunk 2/5
2024-05-23 14:33:46  INFO [ 60%] raft Adding chunk 3/5
2024-05-23 14:34:00  INFO [ 80%] raft Adding chunk 4/5
Saving the dataset (1/1 shards): 100% 10/10 [00:00<00:00, 1296.98 examples/s]
Saving the dataset (1/1 shards): 100% 10/10 [00:00<00:00, 1381.39 examples/s]
Map: 100% 10/10 [00:00<00:00, 1478.06 examples/s]
Creating json from Arrow format: 100% 1/1 [00:00<00:00, 200.39ba/s]


## Format the dataset to finetuning data

Format for chat: https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset

In [ ]:
!python3 format.py --input ../../data-00000-of-00001.arrow --output ../../output.completion.jsonl --output-format chat

with open('../../output.completion.jsonl', 'r') as f:
  data = f.read()
  # Parse the JSONL data into a list of dictionaries
  data = [json.loads(line) for line in data.splitlines()]
  # Print the first dictionary in the list
(data)


Generating train split: 20 examples [00:00, 3251.52 examples/s]
Map: 100% 20/20 [00:00<00:00, 1394.50 examples/s]
Creating json from Arrow format: 100% 1/1 [00:00<00:00, 102.60ba/s]


[{'messages': [{'content': 'Loài rắn nào có thể nhìn tốt?', 'role': 'user'},
   {'content': 'Bước 1: Đoạn văn cho biết: "Chỉ một số loài rắn mới có thể nhìn thấy tốt, nhưng nói chung chúng dựa vào các giác quan khác nhiều hơn là thị giác."\n=> Những loài rắn nhìn tốt là những loài rắn nằm trong số ít đó.\n\n<ANSWER>: Không có thông tin cụ thể về loài rắn nào có thể nhìn tốt, nhưng nhìn chung, chỉ một số ít loài rắn mới nhìn tốt.',
    'role': 'assistant'}]},
 {'messages': [{'content': 'Các loài rắn phụ thuộc vào giác quan nào?',
    'role': 'user'},
   {'content': 'Bước 1: Câu hỏi đã hỏi rằng các loài rắn phụ thuộc vào giác quan nào.\nBước 2: Trong đoạn văn đã cho, có một câu nêu rằng "##begin_quote## nói chung chúng dựa vào các giác quan khác nhiều hơn là thị giác.##end_quote##"\nBước 3: Từ bước 2, ta có thể suy ra rằng các loài rắn phụ thuộc vào các giác quan khác nhiều hơn là thị giác.\n\n<ANSWER>: Các loài rắn phụ thuộc vào các giác quan khác như khứu giác, xúc giác và cảm biến nhi

In [ ]:
from datasets import Dataset

ds = Dataset.from_file("../../gdrive/MyDrive/LLM01 - RắnGPT/Code/RAFT Dataset/-checkpoints-14/data-00000-of-00001.arrow")
print(ds)
dset = ds.to_pandas()

Dataset({
    features: ['id', 'type', 'question', 'context', 'oracle_context', 'cot_answer', 'instruction'],
    num_rows: 30
})
